In [1]:
import gmsh
import subprocess
import math
import sys,os, shutil
import pandas
import numpy as np
gmsh.__file__
import matplotlib.pyplot as plt

In [2]:
# GMSH meshing
# export files for mesh and image of mesh/model
def save_mesh(context, meshpath, gifpath):  
    if  not os.path.isdir(context['project_dir']):
        print(f"Creating project dir: `{context['project_dir']}`")
        print( os.mkdir(context['project_dir']))
        
    if gifpath:
        gmsh.fltk.initialize()
        gmsh.write(os.path.join(context['project_dir'], gifpath))
        gmsh.fltk.finalize()        
    if meshpath:
        gmsh.option.setNumber('Mesh.SaveAll', 1)
        gmsh.write(os.path.join(context['project_dir'],meshpath))

# SU2 Case setup
# build a config for a Solver to run a case:        
def save_config(context, base_config_path='base.cfg'):
    config_path=os.path.join(context['project_dir'], f"{context['project_name']}.cfg")
    project_config=open(base_config_path).read().format(**context)
    print(f"writing CNF: '{config_path}'")
    with open(config_path,'w') as f:
        f.write(project_config)

# build a case - geometry, mesh, physical context, solver params. Maybe should be factored a little...
def create_sim(project_name, fuselage_rad=gmsh.pi/16.0, wing_rad=gmsh.pi/4.0, wedge_rad=gmsh.pi/16.0, 
               meshing=True, 
                xshift=0, camX=30.0,camY=60.0,camZ=0.0,camZoom=5.0,
                meshpath=None, gifpath=None, popup=True, angle_of_attack=0.0, meshsize_large=5.0,
              clip_wing=True, case_index=None, case_name=None, project_dir=None):

    # Setup Geometry 
    p_height = 11.0
    p_width = 8.5
    p_thickness = 0.2
    
    gmsh.initialize()
    gmsh.model.add(project_name)
    gmsh.option.setNumber("General.Trackball", 0)

    gmsh.option.setNumber("General.RotationX", camX) #187.3729455233209))
    gmsh.option.setNumber("General.RotationY", camY) #276.7547707531851))
    gmsh.option.setNumber("General.RotationZ", camZ) # 296.8673379509896))
    gmsh.option.setNumber("General.ScaleX", camZoom) 
    gmsh.option.setNumber("General.ScaleY", camZoom) 
    gmsh.option.setNumber("General.ScaleZ", camZoom) 

    # Actual model!
    phi = fuselage_rad # wedge_radians #gmsh.pi/16.0

    # x coordinate of wing fold
    fuselage_height = p_height* math.sin(phi)

    # right vertical wedge; build then tilt; RF Right Fuselage [a,b,c]
    rfb = [0.0, 0.0, 0.0] # back corner of fuselage; origin
    rfa = [0.0, fuselage_height, 0.0]
    rfc = [p_height, 0.0, 0.0] # tip of front
    
    # right wing, vertical
    rwb = rfa.copy()
    rwa = rwb.copy()
    rwa[1] += p_width/2.0 - fuselage_height

    rwc = rfc.copy()

    rf=make_triangle(rfa, rfb, rfc, 0.2)
    rw=make_triangle(rwa, rwb, rwc, 0.2)    
    
    # trim wing
    if clip_wing:
        cutbox=gmsh.model.occ.addBox(
           rwa[0], p_width/2.0-rfa[1], rfa[2]-5*p_thickness,
           p_height,   p_width, 10*p_thickness )   
        rw, v1b = gmsh.model.occ.cut(rw,[(3,cutbox)], removeTool=True)

    #rotate wing along top edge of fuselage
    gmsh.model.occ.rotate(rw,rfa[0],rfa[1],rfa[2],rfc[0]-rfa[0],rfc[1]-rfa[1],rfc[2]-rfa[2],wing_rad)
    
    # merge fuselage and wing
    rh, rhb = gmsh.model.occ.fuse(rf,rw)
    
    #rotate halfplane along bottom of plane
    gmsh.model.occ.rotate(rh,rfb[0],rfb[1],rfb[2],rfc[0]-rfb[0],rfc[1]-rfb[1],rfc[2]-rfb[2],wedge_rad)
        
    lf=make_triangle(rfa, rfb, rfc, 0.2)
    lw=make_triangle(rwa, rwb, rwc, 0.2)
    
    if clip_wing:
        cutbox2=gmsh.model.occ.addBox(
            rwa[0], p_width/2.0-rfa[1], rfa[2]-5*p_thickness,
           p_height,   p_width, 10.0*p_thickness )   
        lw, v1b = gmsh.model.occ.cut(lw,[(3,cutbox2)])
    
    #rotate wing along top edge of fuselage
    gmsh.model.occ.rotate(lw,rfa[0],rfa[1],rfa[2],rfc[0]-rfa[0],rfc[1]-rfa[1],rfc[2]-rfa[2],-wing_rad)
    
    # merge fuselage and wing
    #print(f"Fuselage: {rf} Wing: {rw}")
    lh, lhb = gmsh.model.occ.fuse(lf,lw)
    
    #rotate halfplane along bottom of plane
    gmsh.model.occ.rotate(lh,rfb[0],rfb[1],rfb[2],rfc[0]-rfb[0],rfc[1]-rfb[1],rfc[2]-rfb[2],-wedge_rad)
    
    p1, p1b =  gmsh.model.occ.fuse(lh,rh)
    
    # build skybox
    b1=gmsh.model.occ.addBox(-4*p_height,-4*p_width,-4*p_width, 8*p_height, 8*p_width, 8*p_width)

    # create negative volume around plane model
    v1, v1b = gmsh.model.occ.cut([(3,b1)],p1)

    gmsh.model.occ.synchronize()

    if meshing:
        lcar_big = meshsize_large
        lcar2 = 1.0
        lcar3 = .055

        # Assign a mesh size to all the points:
        gmsh.model.mesh.setSize(gmsh.model.getEntities(0), lcar_big)
        #gmsh.model.mesh.setSize([(2,s) for s in wall_surfaces], lcar1)
        gmsh.model.mesh.generate(3)

    plane_surfaces = []
    wall_surfaces = []

    surfaces = gmsh.model.occ.getEntities(dim=2)
    for s in surfaces:
        com = gmsh.model.occ.getCenterOfMass(s[0], s[1])
        if np.allclose(com, [p_height/2.0, 0, 0],atol=6.0):
            plane_surfaces.append(s[1])
        else:
            wall_surfaces.append(s[1])

    # Set up physical groups for boundaries and markers in Simulation        
    wall_marker = 1
    plane_marker = 2
    gmsh.model.addPhysicalGroup(2, wall_surfaces, wall_marker)
    gmsh.model.setPhysicalName(2, wall_marker, "Walls")
    gmsh.model.addPhysicalGroup(2, plane_surfaces, plane_marker)
    gmsh.model.setPhysicalName(2, plane_marker, "Plane")

    save_mesh(locals(),meshpath, gifpath)
    save_config(context=locals())
    if popup:
        gmsh.fltk.run()
    gmsh.finalize()

# SU2 Geometry
# Build a triangular volume    
def make_triangle(rp0,rp1,rp2,d, mesh_size=1.0, recenter=True):
    #print(f"making TRI: {rp0} | {rp1} | {rp2}...")
    p1 = gmsh.model.occ.addPoint(rp0[0], rp0[1], rp0[2], mesh_size)
    p2 = gmsh.model.occ.addPoint(rp1[0], rp1[1], rp1[2], mesh_size)
    p3 = gmsh.model.occ.addPoint(rp2[0], rp2[1], rp2[2], mesh_size)
    l1 = gmsh.model.occ.addLine(p1,p2)
    l2 = gmsh.model.occ.addLine(p2,p3)
    l3 = gmsh.model.occ.addLine(p3,p1)
    c1 = gmsh.model.occ.addCurveLoop([l1,l2,l3])
    s1 = gmsh.model.occ.addPlaneSurface([c1])
    v1 = gmsh.model.occ.extrude([(2,s1)], 0.0,0.0,d)
    newVolTag = [x for x in v1 if x[0]==3]
    if recenter:
         gmsh.model.occ.translate(newVolTag, 0, 0, -d/2.0)
    #
    return newVolTag

# SU2 Execution
def run_conf(conf, i, meshing, run, as_row=True, popup=False, extract=['lift', 'drag']):
    case_name=f"{project_name}-{i:0>3}"
    #i = conf['case_index']
    #meshsize = conf['mesh_size']
    meshpath=f"{case_name}.su2"
    gifpath=f"{project_name}-{i:0>3}.jpg"
    create_sim(case_name, meshing=meshing, 
        gifpath=gifpath, 
        meshpath=meshpath if meshing else None,
        popup=popup, #meshsize_large=meshsize,
        **conf)
    if run:
        subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{conf['project_dir']}/{case_name}.cfg"]) 
    run_dict =  {'meshpath': meshpath, 'gifpath':gifpath, 
                 'surface_flow_path': f"{case_name}-surface_flow.csv"}
    if extract:
        sdf=pandas.read_csv( os.path.join(conf['project_dir'],run_dict['surface_flow_path']))
        extracts = {
            'lift':float(sdf[['Momentum_y']].sum()),
            'drag':float(sdf[['Momentum_x']].sum())
        }
        run_dict.update(extracts)
    
    if as_row:
        return pandas.Series([run_dict[c] for c in ['meshpath', 'gifpath', 'surface_flow_path','lift','drag']])
    else:
        return run_dict

def run_confs(conf_list, meshing, run, meshsize=5.0, popup=False, animation=False):

    for i in range(len(conf_list)):
        run_conf(conf_list[i], i, meshing)

    if animation:
        ffmpeg_path='/Users/scot/bin/ffmpeg'
        subprocess.run([ffmpeg_path, "-r", "6", "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}.gif"]) 

# Create and run simulations
### Create sequence of cases

Isn't this where we make a dataframe? Yes. Yes it is.

Dataframes should be (only) information we wouldn't change to run a simulation on a different platform.
So, things like design parameters and case names.
Not things like paths.
Really? 

So things like `project_path` are metadata, associated with the DF not rows. Any given platform will want to build full paths itself with case anmes and such.

In [3]:
project_name = "f5"
project_dir = project_name
df = pandas.DataFrame([{
    "meshsize_large": 5.0,
    "case_index": 5*i + j,
    "case_name": f"{project_name}-{5*i+j:0>3}",
    "project_dir": project_dir,
    "wedge_rad" : gmsh.pi /6.0,
    "wing_rad" : gmsh.pi/2.0,
    "fuselage_rad" : gmsh.pi/48.0 + 0.02*i,
    "angle_of_attack" : -5.0 - j
        } for i in range(5) for j in range(5)])


In [7]:
df

,meshsize_large,case_index,case_name,project_dir,wedge_rad,wing_rad,fuselage_rad,angle_of_attack
0,5.0,0,f5-000,f5,0.523599,1.570796,0.06545,-5.0
1,5.0,1,f5-001,f5,0.523599,1.570796,0.06545,-6.0
2,5.0,2,f5-002,f5,0.523599,1.570796,0.06545,-7.0
3,5.0,3,f5-003,f5,0.523599,1.570796,0.06545,-8.0
4,5.0,4,f5-004,f5,0.523599,1.570796,0.06545,-9.0
5,5.0,5,f5-005,f5,0.523599,1.570796,0.08545,-5.0
6,5.0,6,f5-006,f5,0.523599,1.570796,0.08545,-6.0
7,5.0,7,f5-007,f5,0.523599,1.570796,0.08545,-7.0
8,5.0,8,f5-008,f5,0.523599,1.570796,0.08545,-8.0
9,5.0,9,f5-009,f5,0.523599,1.570796,0.08545,-9.0


In [4]:
d2r=2.0*gmsh.pi/360.0
2.0*d2r

0.03490658503988659

In [9]:
# Functions to setup and run case (SU2 is implicit)
# build cases with no mesh or sim
geometry_only = lambda x: run_conf(x,x['case_index'], True, False, extract=False, popup=True)
# mesh and simulate
full_sim = lambda x: run_conf(x,x['case_index'], True, True)

In [10]:

df[['mesh', 'image', 'surface', 'lift', 'drag']] = df.apply(geometry_only, axis=1)
#df[['mesh', 'image', 'surface', 'lift', 'drag']] = df.apply(geometry_only, axis=1)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 20%] Meshing curve 22 (Line)
Info    : [ 30%] Meshing curve 23 (Line)
Info    : [ 30%] Meshing curve 24 (Line)
I

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : MacOSARM-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] MathEx[contrib] Med Mesh Metis[contrib] Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TouchBar Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.17.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Writing 'f5/f5-000.jpg'...
Info    : Done writing 'f5/f5-000.jpg'
Info    : Writing 'f5/

KeyError: 'lift'

In [ ]:
df.to_csv("f5/f5_df.csv")

In [ ]:
df = pandas.read_csv("f4_df.csv")

In [ ]:
%matplotlib notebook
plt.scatter(df[['fuselage_rad']], df[['lift']])
plt.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits import mplot3d
fig = plt.figure(figsize =(10, 10))
ax = plt.axes(projection ='3d')
 
# Creating plot
ax.scatter(df[['angle_of_attack']],df[['fuselage_rad']], df[['lift']])
ax.set_xlabel("Angle of Attack")
ax.set_ylabel("Fuselage Fold Angle")
ax.set_zlabel("Lift")
ax.set_title("Paper Airplane Design Space")
dir(ax)
plt.show()

In [ ]:
df

## Animate design "slice" in geometry space
it would be nice to move along various paths through this design space (eg, hold AoA and move the Fuselage Angle across an interesting range) and visualize the change in design and flow with a simple animation.

We'll define the slive of the DF we want, then extract the individual case outputs for them. We can generate the viz for each individually, then export them to frames in an animation.

This is weird because my Paraview python and my Jupyter/GMSH python aren't friends. So, we read a base script to do the viz we want, parametrize in any needed values from the DF, then write out a new python file ready to be run by 

In [ ]:
def visualize_case(project_dir, case_name, base_vis_path="para.py"):
    
    base_vis = open(base_vis_path).read()
    #lame_name_hack:
    #proj, idx = case_name.split('_',1)
    #case_name = f"{proj}-{idx:0>3}"
    # interpolated_vis = base_vis.format(locals())
    # case_vis_path=path.join(project_dir, f"{case_name}-para.py")
    #with open(case_vis_path,'w') as f:
    #    f.write(interpolated_vis)
    pvpath = "/Applications/ParaView-5.11.0.app/Contents/bin/pvpython" # para.py fold-004
    print(f"Vis proc for {case_name}...")
    subprocess.run([pvpath, base_vis_path, f"{project_dir}/{case_name}"])
    return f"/Users/scot/Projects/{project_dir}/{case_name}-para.png"
    
vis_path = lambda x: visualize_case(x['project_dir'], x['case_name'])

def animate(project_name, file_paths, output_dir, framerate="2"):
    ffmpeg_path='/Users/scot/bin/ffmpeg'
    for i,file_path in enumerate(file_paths):
        shutil.copyfile(file_path, os.path.join(output_dir, f"animation_frame-{i:0>3}.png"))
    subprocess.run([ffmpeg_path, "-r", framerate,  "-y", "-i", 
                    f"/Users/scot/Projects/{project_name}/animation_frame-%03d.png", 
                    f"/Users/scot/Projects/{project_name}/{project_name}.gif"]) 
    #for i,file_path in enumerate(file_paths):
    #    os.remove(os.path.join(output_dir, f"animation_frame_{i}.png"))
    return f"/Users/scot/Projects/{project_name}/{project_name}.gif"

### run visualizations and set path to vis outputs

In [ ]:
df['vis_path'] = df.apply(vis_path,axis=1)

In [ ]:

animate('f4', list(df.loc[df['angle_of_attack']==3.0][['vis_path']]['vis_path']), output_dir='f4')

In [ ]:
visualize_case('f2', 'f2_3')

In [ ]:
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-r", "6", "-i", 
                f"/Users/scot/Projects/{project_name}/{project_name}-%03d.jpg", 
                f"/Users/scot/Projects/{project_name}{project_name}.gif"]) 

### Make exploratory (unmeshed) animation of models

In [ ]:
df['geometry'] = run_confs(conf_list, False, False)

### Generate meshes
would be nice to leave the animation alone...

In [ ]:
for i in range(len(conf_list)):
    case_name=f"{project_name}-{i:0>3}"
    create_sim(case_name, meshing=True, 
                gifpath=f"{project_name}_meshed-{i:0>3}.jpg", 
                meshpath=f"{case_name}.su2",
                popup=False, meshsize_large=2.0,
                **conf_list[i])
    #subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{case_name}.cfg"]) 
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}_meshed-%03d.jpg", f"{project_name}_meshed.gif"]) 

## Actually run cases!

In [ ]:
for i in range(len(conf_list)):
    case_name=f"{project_name}-{i:0>3}"
    create_sim(case_name, meshing=True, 
                gifpath=None, #f"{project_name}-{i:0>3}.jpg", 
                meshpath=f"{case_name}.su2",
                popup=False, meshsize_large=4.0,
                **conf_list[i])
    subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", f"{case_name}.cfg"]) 
#ffmpeg_path='/Users/scot/bin/ffmpeg'
#subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}_meshed.gif"]) 

In [ ]:
import subprocess
        # subprocess.call("ffmpeg -i t8-%d.jpg t8.mpg".split(' '))
# /Users/scot/bin/ffmpeg -i /Users/scot/Projects/pplane-%03d.jpg test.gif
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-i", f"/Users/scot/Projects/{project_name}-%03d.jpg", f"{project_name}.gif"]) 

### Superlight config formatting

In [ ]:
base_config_path='base.cfg'
config_path=f"{project_name}.cfg"
project_config=open(base_config_path).read().format(**globals())
with open(config_path,'w') as f:
    f.write(project_config)
#base_config

## Run subprocess
* Used to need this for `gmsh` file translation, but no longer.
* Now is this just to run SU2? Fine, but likely replaceable with a direct SU2 API....

In [ ]:
import subprocess

subprocess.run(["/Users/scot/Projects/SU2/bin/SU2_CFD", config_path]) 

## Read field output from Simulation run

In [ ]:
import pandas
for i in range(10):
    sdf=pandas.read_csv(f"fold-{i:0>3}-surface_flow.csv")
    print(f"{conf_list[i]['fuselage_rad']}: {df[['Momentum_y']].sum()} {df[['Momentum_x']].sum()}")

In [ ]:
sys.path.append('/Applications/ParaView-5.11.0.app/Contents/Python/')
import paraview.simple
# '/Applications/ParaView-5.11.0.app/Contents/Python/

In [ ]:
ffmpeg_path='/Users/scot/bin/ffmpeg'
subprocess.run([ffmpeg_path, "-r", "6", "-i", f"/Users/scot/Projects/{project_name}-%03d-para.png", f"{project_name}-para.gif"]) 